## Selective Search 예제

### 코드 다운로드

In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/opencv-selective-search/opencv-selective-search.zip
!unzip -qq opencv-selective-search.zip
%cd opencv-selective-search

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import random

In [ ]:
def plt_imshow(title, image):
    # convert the image frame BGR to RGB color space and display it
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(title)
    plt.grid(False)
    plt.show()

### segmetation 데모

In [ ]:
def graph_segmentation_demo(image_path, sigma=0.5, k=300, min_size=1000):
    # Load the image
    image = cv2.imread(image_path)

    # Create the GraphSegmentation object
    gs = cv2.ximgproc.segmentation.createGraphSegmentation()

    # Set the sigma, k, and min_size parameters
    gs.setSigma(sigma)
    gs.setK(k)
    gs.setMinSize(min_size)

    # Process the image
    # print(image.shape)
    segments = gs.processImage(image)
    # print(segments.shape)

    # Create a color image to visualize the segmentation
    visual = np.zeros((segments.shape[0], segments.shape[1], 3), dtype=np.uint8)
    segment_ids = np.unique(segments)
    # print(segment_ids)
    for segment_id in segment_ids:
        visual[segments == segment_id] = [np.random.randint(0, 256) for _ in range(3)]

    return visual

In [ ]:
image_path = "dog.jpg"
visual = graph_segmentation_demo(image_path,0.1,2000,100)
plt_imshow("seletive search", visual)

### selective search를 이용한 region proposal

In [ ]:
def selective_search_demo(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Create a Selective Search Segmentation Object using the default graph-based model
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)


    # Choose the segmentation method (fast or quality)
    ss.switchToSelectiveSearchFast()

    # Run selective search and get the region proposals
    rects = ss.process()
    print(f"Total Region Proposals: {len(rects)}")
    print(rects)

    # Visualize a subset of region proposals
    num_show_rects = 100
    clone = image.copy()
    for i, rect in enumerate(rects):
        if i < num_show_rects:
            x, y, w, h = rect
            cv2.rectangle(clone, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            break

    return clone

In [ ]:
image_path = "dog.jpg"
visual = selective_search_demo(image_path)
plt_imshow("seletive search", visual)

### 객체 감지를 위한 OpenCV 선택적 검색

In [ ]:
args = {
    "image": "dog.jpg",
    "method": "fast"
}

In [ ]:
# load the input image
image = cv2.imread(args["image"])

# initialize OpenCV's selective search implementation and set the
# input image
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)

# check to see if we are using the *fast* but *less accurate* version
# of selective search
if args["method"] == "fast":
    print("[INFO] using *fast* selective search")
    ss.switchToSelectiveSearchFast()

# otherwise we are using the *slower* but *more accurate* version
else:
    print("[INFO] using *quality* selective search")
    ss.switchToSelectiveSearchQuality()

In [ ]:
# run selective search on the input image
start = time.time()
rects = ss.process()
end = time.time()

# show how along selective search took to run along with the total
# number of returned region proposals
print("[INFO] selective search took {:.4f} seconds".format(end - start))
print("[INFO] {} total region proposals".format(len(rects)))

In [ ]:
# loop over the region proposals in chunks (so we can better
# visualize them)
for i in range(0, len(rects), 100):
    # clone the original image so we can draw on it
    output = image.copy()

    # loop over the current subset of region proposals
    for (x, y, w, h) in rects[i:i + 100]:
        # draw the region proposal bounding box on the image
        color = [random.randint(0, 255) for j in range(0, 3)]
        cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)

    # show the output image
    plt_imshow("Output", output)